# data collection

In [62]:
# color images 
import cv2
import os
import face_recognition

# Open the video capture
video = cv2.VideoCapture(1)

# Load the face detection classifier (not used in face_recognition)
# facedetect = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# Get user ID
id = input("Enter Your ID: ")

# Create a directory for the person (if it doesn't exist)
person_dir = 'DATASET/' + str(id)
os.makedirs(person_dir, exist_ok=True)

count = 1

while True:
    # Read a frame from the video
    ret, frame = video.read()

    # Detect faces using face_recognition library
    face_locations = face_recognition.face_locations(frame)

    for (top, right, bottom, left) in face_locations:
        count += 1
        # Save the face region as an image
        face_image_path = os.path.join(person_dir, f'{count}.jpg')
        cv2.imwrite(face_image_path, frame[top:bottom, left:right])

        # Draw a rectangle around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (50, 50, 255), 1)

    # Display the frame
    cv2.imshow("Frame", frame)

    # Break the loop if enough images have been collected
    if count > 50:
        break

    # Wait for a key press
    k = cv2.waitKey(1)

# Release the video capture and close all windows
video.release()
cv2.destroyAllWindows()

print("Dataset Collection Done..................")


Enter Your ID: T Hemanth Kumar
Dataset Collection Done..................


# create a model 

In [1]:
import face_recognition
from sklearn import neighbors
import os
import pickle

# Function to train a k-nearest neighbors classifier
def train_knn(train_dir, model_save_path=None, n_neighbors=None, knn_algo='ball_tree'):
    X = []
    y = []

    # Loop through each person in the training set
    for class_dir in os.listdir(train_dir):
        if not os.path.isdir(os.path.join(train_dir, class_dir)):
            continue

        # Loop through each image for the current person
        for img_name in os.listdir(os.path.join(train_dir, class_dir)):
            img_path = os.path.join(train_dir, class_dir, img_name)
            image = face_recognition.load_image_file(img_path)
            faces_bboxes = face_recognition.face_locations(image)

            # If only one face is detected, add it to the training set
            if len(faces_bboxes) == 1:
                X.append(face_recognition.face_encodings(image, known_face_locations=faces_bboxes)[0])
                y.append(class_dir)

    # Train the KNN classifier
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)

    # Save the trained KNN classifier to a file
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf

# Directory where the face images are saved
dataset_dir = 'DATASET'

# Train the KNN classifier using the collected face images
classifier = train_knn(dataset_dir, model_save_path='trained_model.clf', n_neighbors=5)
print("Facial Recognition Model Trained.")


Facial Recognition Model Trained.


# test the model

In [9]:
import cv2
import face_recognition
import pickle
import datetime
import os
import csv
import tkinter as tk
from tkinter import messagebox

# Load the trained KNN classifier
with open('trained_model.clf', 'rb') as f:
    knn_clf = pickle.load(f)

    
# Dictionary to store verification images and their encodings
verification_images = {
    'INDIRA': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\A Indira\15.jpg",
    'ROHITH': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\A Rohit\4.jpg",
    'EKKA ABHI': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\Abhishek ekka\4.jpg",
    'ADITYA': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\B ADITYA\44.jpg",
    'ABHISHIEK': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\G ABHISHEK\42.jpg",
    'HARSHA VARDHAN ': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\B Harsha Vardhan\29.jpg",
    'JITENDRA': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\B Jithendra\45.jpg",
    'PAVAN KUMAR': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\B Pavan Kumar\30.jpg",
    'TAJASWARAO': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\B Tejaswarrao\41.jpg",
    'BHASKAR': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\D BHASKAR\21.jpg",
    'NIHARIKA': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\D Niharika\15.jpg",
    'PURUSHOTHAM': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\D Purushotham\44.jpg",
    'RAMPRASADRAO': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\D Ramprasad rao\18.jpg",
    'SASIKANTH': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\D Sasi Kanth\43.jpg",
    'SNEHITA': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\G Snehitha\41.jpg",
    'SRIYAREDDY': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\G Sriya Reddy\21.jpg",
    'NARENDRA': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\I Narendra\3.jpg",
    'MANIKANTA': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\J Manikanta\31.jpg",
    'JAYARAJU': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\K Jayaraju\41.jpg",
    'VENKATSAI': r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\K Venkata Sai Teja\16.jpg",
    'LAVANYA' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\Konganpalli Lavanya\17.jpg",
    'CHANDINI' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\L Chandini\18.jpg",
    'POJITHA' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\L Poojitha\20.jpg",
    'GURUMONOHAR' :r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\M Guru Manohar\31.jpg",
    'HARINI' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\M Haarini Sree\17.jpg",
    'NAVEEN' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\M Naveen\20.jpg",
    'PREMSAI' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\M RAJA PREM SAI\16.jpg",
    'SAITEJA' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\M Sai Teja\18.jpg",
    'KISHORE' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\P Nanda Kishore\44.jpg",
    'VINOD' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\P Vinod\29.jpg",
    'POJA' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\Pooja Chandwani\30.jpg",
    'DILLESARARAO' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\R Dhilleeswar rao\4.jpg",
    'SAICHARAN' :  r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\R Sai Charan\15.jpg",
    'RASOOL' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\S K M Rasool\7.jpg",
    'MADHURI' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\S Madhuri\18.jpg",
    'NITISH' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\S Madhuri\18.jpg",
    'SAMEERA' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\S SAMEERA\39.jpg",
    'SAIKIRAN' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\S.sai kiran\4.jpg",
    'HEMANTH' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\T Hemanth Kumar\8.jpg",
    'SAISANDESH' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\T Jai Sandesh\6.jpg",
    'PRASANNA' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\T Prasanna\42.jpg",
    'SATYANARANAYA' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\T Satyanarayana\18.jpg",
    'VIVEKANANDHA' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\T Vivekananda\31.jpg",
    'ROHAN' : r"C:\Users\vijay\ABHISHEK JR FACE ATTENDENCE PROJECT SEP\DATASET\Y ROHAN KUMAR(221801350001)\6.jpg"
    
    # Add more persons as needed
}

# Load verification images and their encodings
verification_encodings = {}
for person, image_path in verification_images.items():
    verification_image = face_recognition.load_image_file(image_path)
    # Ensure that at least one face is detected
    face_encodings = face_recognition.face_encodings(verification_image)
    if face_encodings:
        verification_encodings[person] = face_encodings[0]
    else:
        print(f"No face detected in {person}'s image: {image_path}")

# Open the video capture
video = cv2.VideoCapture(0)

# CSV file to record recognition and verification events
csv_file_path = 'recognition_log.csv'

# Folder to save all recognized faces
output_folder = 'recognized_faces'
os.makedirs(output_folder, exist_ok=True)

# File to save recognized names
recognized_names_file_path = 'recognized_names.txt'

# Metrics variables
start_time = None
total_recognitions = 0
average_recognition_time = 0.0
confidence_threshold = 0.5  # Set the confidence threshold as needed

# CSV columns
csv_columns = ['Timestamp',   'Year',   'Month', 'Date', 'Hour', 'Minute', 'Second', 'Name']

with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_columns)
    writer.writeheader()

while True:
    # Read a frame from the video
    ret, frame = video.read()

    # Detect faces using face_recognition library
    face_locations = face_recognition.face_locations(frame)

    # Ensure that there are face encodings
    if len(face_locations) > 0:
        face_encodings = face_recognition.face_encodings(frame, known_face_locations=face_locations)

        # Perform face verification
        for face_encoding, face_location in zip(face_encodings, face_locations):
            name = knn_clf.predict([face_encoding])[0]
            (top, right, bottom, left) = face_location

            # Perform face recognition with confidence
            face_distances = face_recognition.face_distance(list(verification_encodings.values()), face_encoding)
            min_distance = min(face_distances)
            if min_distance < confidence_threshold:
                recognized_name = list(verification_encodings.keys())[list(face_distances).index(min_distance)]
            else:
                recognized_name = "Unknown"

            # Log recognition event
            with open(csv_file_path, 'a', newline='') as csv_file:
                timestamp = datetime.datetime.now()
                writer = csv.DictWriter(csv_file, fieldnames=csv_columns)
                writer.writerow({
                    'Timestamp': timestamp.strftime('%Y-%m-%d %H:%M:%S'),
                    'Year': timestamp.year,
                    'Month': timestamp.month,
                    'Date': timestamp.day,
                    'Hour': timestamp.hour,
                    'Minute': timestamp.minute,
                    'Second': timestamp.second,
                    'Name': recognized_name
                })

            # Display the name of the recognized person
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, f"{recognized_name}", (left + 6, bottom - 6), font, 0.5, (255, 255, 0), 1)  # Yellow text

            # Draw a rectangle around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

            if recognized_name != "Unknown":
                # Save the recognized face to the output folder
                face_image = frame[top:bottom, left:right]
                face_filename = os.path.join(output_folder,
                                             f"{recognized_name}_{timestamp.strftime('%Y%m%d%H%M%S')}.jpg")
                cv2.imwrite(face_filename, face_image)

                # Save the recognized name to the text file
                with open(recognized_names_file_path, 'a') as names_file:
                    names_file.write(f"{recognized_name}\n")

                # Display pop-up message
                root = tk.Tk()
                root.withdraw()
                messagebox.showinfo("Attendance Taken", f"Attendance taken for {recognized_name}")

                # Update metrics
                total_recognitions += 1
                if start_time is not None:
                    elapsed_time = (datetime.datetime.now() - start_time).total_seconds()
                    average_recognition_time = (
                            average_recognition_time * (total_recognitions - 1) + elapsed_time) / total_recognitions

            # Record start time for the next face
            start_time = datetime.datetime.now()

    # Display metrics on the frame
    cv2.putText(frame, f"Total Recognitions: {total_recognitions}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                (0, 255, 255), 2)  # Yellow text
    cv2.putText(frame, f"Average Recognition Time: {average_recognition_time:.2f} seconds", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)  # Yellow text

    # Display the frame
    cv2.imshow("Face Recognition and Verification", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
video.release()
cv2.destroyAllWindows()

# stream lit

In [4]:
import streamlit as st
import pandas as pd

# Load the CSV file
csv_file_path = 'recognition_log.csv'
df = pd.read_csv(csv_file_path)

# Streamlit app
st.title('Face Recognition Log Viewer')

# Display the dataframe
st.write(df)

2024-04-17 20:38:40.097 
  command:

    streamlit run C:\Users\vijay\anaconda3\vijaykumar\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [5]:
!streamlit run face.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: face.py
